In [1]:
import pandas as pd
import numpy as np
from sklearn import neighbors
import matplotlib.pyplot as ptl
from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH


In [2]:
trip = pd.read_csv('data/trip_train.csv')

In [3]:
trip.sample(2)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
252837,17142,347,9/10/2013 19:08,Embarcadero at Folsom,51,9/10/2013 19:13,Yerba Buena Center of the Arts (3rd @ Howard),68,539,Subscriber,94107
209328,337525,239,6/24/2014 16:55,Beale at Market,56,6/24/2014 16:59,Commercial at Montgomery,45,259,Subscriber,94133


weather.columns

In [4]:
def splitDateAndTime(dateAndTime):
    splitLine = dateAndTime.split(' ')
    return str(splitDate(splitLine[0]))  + splitTime(splitLine[1])

def splitDate(date):
    lineSplit = date.split('/')
    return str(lineSplit[2]) + str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1] if int(lineSplit[1]) >= 10 else '0' + str(lineSplit[1]) )

def splitTime(time):
    lineSplit = time.split(':')
    return str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1])


weather.sample(2)

In [5]:
modifTrip = trip[['start_date','duration','start_station_name','start_station_id','subscription_type']]

· Agregar columna para dia de la semana

· Agregar columna para hora del dia

· Agregar columna para si es festividad o no

· Agregar columna para el mes

El formato de fecha esta dado por: '201512251404', por lo que se van a tomar los ultimos 4 numeros

In [6]:
modifTrip['time'] = modifTrip['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )

/home/brahvic/.local/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
modifTrip.loc[:,'start_date'] = modifTrip['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
modifTrip.sample(1)

,start_date,duration,start_station_name,start_station_id,subscription_type,time
530616,2015-06-23 08:04:00,29919,San Francisco Caltrain (Townsend at 4th),70,Subscriber,804


In [9]:
modifTrip.loc[:,'day_of_week'] = modifTrip['start_date'].apply(lambda x : x.dayofweek )

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [10]:
modifTrip.loc[:,'month'] = modifTrip['start_date'].apply(lambda x : x.month )

In [11]:
modifTrip.loc[:,'year'] = modifTrip['start_date'].apply(lambda x : x.year )

In [12]:
modifTrip.loc[:,'dayofyear'] = modifTrip['start_date'].apply(lambda x : x.dayofyear )

In [13]:
modifTrip.loc[:,'start_date'] = modifTrip['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [14]:
modifTrip.sample(1)

,start_date,duration,start_station_name,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear
468260,2014-06-25,726,Embarcadero at Sansome,60,Subscriber,917,2,6,2014,176


In [15]:
cal = calendarUSFH()
holidays = cal.holidays(return_name=True,start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))
holiday_festive_day = pd.DataFrame(holidays,columns=['holiday']).reset_index()
holiday_festive_day.rename(columns={'index':'start_date'},inplace=True)


In [16]:
holiday_festive_day.sample(1)

,start_date,holiday
6,2013-10-14,Columbus Day


In [17]:
holiday_festive_day.loc[:,'start_date'] = holiday_festive_day['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [18]:
tripsModi = pd.merge(holiday_festive_day,modifTrip,on=['start_date'],how='right')

In [19]:
tripsModi.sample()

,start_date,holiday,duration,start_station_name,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear
536759,2014-08-09,NaN,241,California Ave Caltrain Station,36,Subscriber,0,5,8,2014,221


In [20]:
tripsModi.loc[:,'is_holiday'] = tripsModi.holiday.isnull()

In [21]:
tripsModi.loc[:,'is_holiday'] = tripsModi['is_holiday'].apply(lambda x : not x )
tripsModi.sample(2)

,start_date,holiday,duration,start_station_name,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
538917,2014-07-18,NaN,732,Commercial at Montgomery,45,Subscriber,1134,4,7,2014,199,False
212126,2013-12-09,NaN,845,Temporary Transbay Terminal (Howard at Beale),55,Subscriber,906,0,12,2013,343,False


In [22]:
tripsModi.loc[:,'subscription_type'] = tripsModi['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )
tripsModi.sample()

,start_date,holiday,duration,start_station_name,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
14677,2015-08-07,NaN,1557,Harry Bridges Plaza (Ferry Building),50,1,1152,4,8,2015,219,False


#  Seleccionando nuestro set para pruebas es estimacion error

In [ ]:
target = tripsModi[['duration']]

In [ ]:
train = tripsModi.drop(labels=['start_date','duration','start_station_name','holiday'],axis=1)

In [ ]:
train.sample()

In [ ]:
print help(neighbors.KNeighborsRegressor)

In [ ]:
knn = neighbors.KNeighborsRegressor(n_neighbors=9,n_jobs=-1,p=5 )

In [ ]:
knn.fit(train,target)

In [ ]:
test = tripsModi.sample(5000)

In [ ]:
test_target = test['duration']

In [ ]:
test = test.drop(labels=['start_date','duration','start_station_name','holiday'],axis=1)

In [ ]:
test.sample()

In [ ]:
test_prediction = knn.predict(test)

In [ ]:
predictionDF = pd.DataFrame(test_prediction,columns={"prediction"})

In [ ]:
predictionDF.loc[:,'spectate'] = test_target.values

In [ ]:
predictionDF.sample(5)

In [63]:
def predictError(predict,spectate):
    if (spectate > predict) : return  ((spectate-predict)/spectate)*100
    else  :  return ((predict-spectate)/predict )*100

In [ ]:
predictionDF['errorValue'] = predictionDF.apply(axis=1,func=lambda x : predictError(x[0],x[1]))

In [ ]:
predictionDF.sample(5)

In [ ]:
print 'El error promedio que comete KNN al predecir es de', float(predictionDF['errorValue'].mean()),'%'

In [ ]:
print "El algoritmo KNN predice un ",\
float(predictionDF[predictionDF['errorValue'] < 15].shape[0]) / float(predictionDF.shape[0]) *100 ,\
"% de los valores, con un error menor al 15%"


#  Agregando columnas del clima

In [145]:
modifTrip.sample()

,start_date,duration,start_station_name,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear
194618,2014-10-11,450,Embarcadero at Bryant,54,Subscriber,1807,5,10,2014,284


In [146]:
station = pd.read_csv('data/station.csv')

In [147]:
station.sample()

,id,name,lat,long,dock_count,city,installation_date
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014


In [148]:
weather = pd.read_csv('data/weather.csv')

In [149]:
weather.zip_code.unique()

array([94107, 94063, 94301, 94041, 95113])

In [150]:
stationReduced = station.loc[:,['id','name','city']]

In [151]:
def choseZipCode(city):
    if city == 'San Francisco' : return 94107
    elif city == 'San Jose' : return 95113
    elif city == 'Redwood City' : return 94063
    elif city == 'Palo Alto' : return 94301
    elif city == 'Mountain View' : return 94041

In [152]:
stationReduced['zip_code'] = stationReduced.loc[:,'city'].apply(choseZipCode)

In [153]:
stationReduced = stationReduced.drop(labels=['id','city'],axis=1)

In [154]:
stationReduced.rename(columns={'name':'start_station_name'},inplace=True)


In [155]:
stationReduced.sample()

,start_station_name,zip_code
26,Rengstorff Avenue / California Street,94041


In [156]:
tripsModifWithZipCode = pd.merge(tripsModi,stationReduced,on=['start_station_name'],how='right')

In [157]:
tripsModifWithZipCode.sample()

,start_date,holiday,duration,start_station_name,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday,zip_code
199902,2014-07-30,NaN,460,2nd at Townsend,61,0,937,2,7,2014,211,False,94107


In [158]:
weather.sample()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
429,10/29/2013,63.0,56.0,48.0,48.0,45.0,42.0,80.0,66.0,52.0,...,10.0,10.0,15.0,4.0,16.0,0,5.0,NaN,355.0,94301


In [159]:
weather['date'] = weather['date'].apply(lambda date : pd.to_datetime(date).date())

In [160]:
weather.dtypes

date                               object
max_temperature_f                 float64
mean_temperature_f                float64
min_temperature_f                 float64
max_dew_point_f                   float64
mean_dew_point_f                  float64
min_dew_point_f                   float64
max_humidity                      float64
mean_humidity                     float64
min_humidity                      float64
max_sea_level_pressure_inches     float64
mean_sea_level_pressure_inches    float64
min_sea_level_pressure_inches     float64
max_visibility_miles              float64
mean_visibility_miles             float64
min_visibility_miles              float64
max_wind_Speed_mph                float64
mean_wind_speed_mph               float64
max_gust_speed_mph                float64
precipitation_inches               object
cloud_cover                       float64
events                             object
wind_dir_degrees                  float64
zip_code                          

In [161]:
tripsModifWithZipCode.rename(columns={'start_date':'date'},inplace=True)

In [162]:
tripsModifWithZipCode.sample()

,date,holiday,duration,start_station_name,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday,zip_code
102881,2013-10-26,NaN,1310,South Van Ness at Market,66,1,1604,5,10,2013,299,False,94107


In [172]:
sampleOfMerge = pd.merge(tripsModifWithZipCode,weather,on=['date','zip_code'],how='left')

In [173]:
sampleOfMerge = sampleOfMerge.drop(labels=['precipitation_inches','events',],axis=1)
sampleOfMerge.shape

(540635, 33)

In [175]:
sampleOfMerge = sampleOfMerge.dropna()

In [176]:
sampleOfMerge.shape

(7829, 33)

##  Corregir columnas con Nan

In [168]:
target = sampleOfMerge[['duration']]

In [53]:
train = sampleOfMerge.drop(labels=['date','duration','holiday','start_station_name'],axis=1)

In [140]:
knn7p5 = neighbors.KNeighborsRegressor(n_neighbors=12,n_jobs=-1,p=2 )

In [141]:
knn7p5.fit(train,target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=12, p=2,
          weights='uniform')

In [144]:
sampleOfMerge.shape

(7829, 33)

In [143]:
test = sampleOfMerge.sample(5000)

In [128]:
test_target = test['duration']

In [129]:
test = test.drop(labels=['date','duration','start_station_name','holiday'],axis=1)

In [130]:
test_prediction = knn7p5.predict(test)

In [131]:
predictionDF = pd.DataFrame(test_prediction,columns={"prediction"})

In [132]:
predictionDF.loc[:,'spectate'] = test_target.values

In [133]:
predictionDF['errorValue'] = predictionDF.apply(axis=1,func=lambda x : predictError(x[0],x[1]))

In [134]:
predictionDF.sample(5)

,prediction,spectate,errorValue
3891,2732.416667,3019,9.492658
3317,619.083333,215,65.271234
4075,976.666667,548,43.890785
4322,1864.416667,17168,89.140164
2503,850.333333,910,6.556777


In [135]:
print 'El error promedio que comete KNN al predecir es de', float(predictionDF['errorValue'].mean()),'%'

El error promedio que comete KNN al predecir es de 44.0367982334 %


In [136]:
print "El algoritmo KNN predice un ",\
float(predictionDF[predictionDF['errorValue'] < 15].shape[0]) / float(predictionDF.shape[0]) *100 ,\
"% de los valores, con un error menor al 15%"

El algoritmo KNN predice un  17.14 % de los valores, con un error menor al 15%
